Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Homework 7: Counting Stacks and Queues
### Copyright Luca de Alfaro, 2019-21.  License: CC-BY-NC. 

## Submission

* Download the .ipynb file:

    File > Download .ipynb
    
    
* Then, [submit the .ipynb to this Google Form](https://docs.google.com/forms/d/e/1FAIpQLSd1xb4Pj7tK6HWOciXwM4Twtf2eYY716j2wqcY1HHyLzlbOvQ/viewform?usp=sf_link).  This homework is due by **11:59pm on Tuesday, 1 February 2022**.


## About this homework

Two things are new about this homework assignment. 

First, this homework is graded using `assert`, which is the standard way in Python of asking for something to be True.  So, a cell succeeds if no error is raised; you no longer see "Success" printed out all the time. 

Second, there are essentially no hidden tests: rather, we make sure your implementation works by comparing your implementation with a reference one.  Precisely, we will be comparing the implementation of CountingQueue with that of Queue. 

## Stacks, Queues, and Their Counting Versions

A stack is a data structure with two operations: push, and pop.  Picture it as a pile of dishes sitting on a counter.  A push operation places a dish on top of the pile.  A pop operation returns the dish on top of the pile, or None if the pile is empty, that is, contains no dishes.  A "dish" can be any Python object. 

A queue is a data structure with two operations: put, and get.  Imagine it as a stack of books horizontally on a shelf.  A put operation adds the book to the left end of the books on the shelf; a get operation gets the book from the right end of the shelf.  

Thus, the difference between a stack and a queue is that the stack is FILO (First In, Last Out), whereas the queue is FIFO (First In, First Out).  Elements in a stack are retrieved newest first. 
Elements in a queue are retrieved in the order they were put in, oldest first.

We will implement here these data structures, with a small twist: we will also introduce _counting_ versions of them, which avoid keeping multiple identical copies of objects in a row. 



Let us begin by implementing a plain vanilla stack.

In [ ]:
class Stack(object):

    def __init__(self):
        self.stack = []

    def __repr__(self):
        """Defining a __repr__ function will enable us to print the
        stack contents, and facilitate debugging."""
        return repr(self.stack) # Good enough.

    def push(self, x):
        """The "top" of the stack is the end of the list."""
        self.stack.append(x)

    def pop(self):
        return self.stack.pop() if len(self.stack) > 0 else None

    def isempty(self):
        return len(self.stack) == 0

    def __len__(self):
        return len(self.stack)

    def __iter__(self):
        for el in self.stack:
            yield el

    def __getitem__(self, i):
        return self.stack[i]

    def __contains__(self, x):
        return x in self.stack



Let's see how this works.

In [ ]:
s = Stack()
print(s.pop())
s.push('a')
s.push('b')
print(s.pop())
print(s.pop())
print(s.pop())


None
b
a
None


Ok!  The definition of a queue is similar. 

In [ ]:
class Queue(object):

    def __init__(self):
        self.queue = []

    def __repr__(self):
        """Defining a __repr__ function will enable us to print the
        queue contents, and facilitate debugging."""
        return repr(self.queue) # Good enough.

    def add(self, x):
        self.queue.append(x)

    def get(self):
        # This is the only difference compared to the stack above.
        return self.queue.pop(0) if len(self.queue) > 0 else None

    def isempty(self):
        return len(self.queue) == 0

    def __len__(self):
        return len(self.queue)

    def __iter__(self):
        for el in self.queue:
            yield el

    def __getitem__(self, i):
        return self.queue[i]

    def __contains__(self, x):
        return x in self.queue


Let's see how it works. 

In [ ]:
s = Queue()
print(s.get())
s.add('a')
s.add('b')
print(s.get())
print(s.get())
print(s.get())


None
a
b
None


As you see, in a queue, the elements are retrieved in the same order in which they were added. 

Python experts might note that, for a queue, we would do better by using the [`collections.deque` class](https://docs.python.org/3.7/library/collections.html#collections.deque), rather than the list class, to make the `pop(0)` operation more efficient; in lists, it takes time proportional to the length of the list; in deques, it takes constant time.  For small lists, however, the difference is negligible.


We now consider a use case in which we may need to put in the queue or stack many repeated copies of the same object.  For instance, assume that the queue is used to store events, and assume that some event may end up being repeated many times in a row.  As an example, the events can be "s", for the tick of a second, "m", when the minute advances, and "h", when the hour advances.  There will be 60 consecutive "s" events between any two "m" events, and it seems a waste to store so many consecutive identical events.  Storing many identical things in a row is akin to counting in unary notation, after all.  We would be better off storing the repeated elements only once, along with a count of the number of times they occur.  Let's develop a queue using this idea (a stack can be done similarly).

In [ ]:
class CountingQueue(object):

    def __init__(self):
        self.queue = []

    def __repr__(self):
        return repr(self.queue)

    def add(self, x, count=1):
        # If the element is the same as the last element, we simply
        # increment the count.  This assumes we can test equality of
        # elements.
        if len(self.queue) > 0:
            xx, cc = self.queue[-1]
            if xx == x:
                self.queue[-1] = (xx, cc + count)
            else:
                self.queue.append((x, count))
        else:
            self.queue = [(x, count)]

    def get(self):
        if len(self.queue) == 0:
            return None
        x, c = self.queue[0]
        if c == 1:
            self.queue.pop(0)
            return x
        else:
            self.queue[0] = (x, c - 1)
            return x

    def isempty(self):
        # Since the count of an element is never 0, we can just check
        # whether the queue is empty.
        return len(self.queue) == 0


Let's put this to the same test as before, printing the queue contents at each step to see what is going on.

In [ ]:
q = CountingQueue()
q.add('a')
print(q)
q.add('b', count=5)
print(q)
q.add('c', count=2)
print(q)
while not q.isempty():
    print(q.get())
    print(q)


[('a', 1)]
[('a', 1), ('b', 5)]
[('a', 1), ('b', 5), ('c', 2)]
a
[('b', 5), ('c', 2)]
b
[('b', 4), ('c', 2)]
b
[('b', 3), ('c', 2)]
b
[('b', 2), ('c', 2)]
b
[('b', 1), ('c', 2)]
b
[('c', 2)]
c
[('c', 1)]
c
[]


It works!  And notice that it works even if we add elements one by one.

In [ ]:
q = CountingQueue()
for i in range(10):
    q.add('a')
q.add('b')
for i in range(3):
    q.add('c', count=2)
print(q)


[('a', 10), ('b', 1), ('c', 6)]


## The Homework Assignment

For this homework, you must implement the following methods for `CountingQueue`: 

* `__len__`
* `__iter__`
* `__in__`
* `__getitem__`

Your goal is to have `CountingQueue` behave exactly like `Queue` to an outside user: the objects have to be different only due to their internal implementation.  So for instance, `__len__` must return the number of elements, including repetitions; not the number of (element, count) pairs in `self.queue`. 

Note that we are adding methods to a class that has already been defined, so our definition have the following somewhat unusual form: 

In [ ]:
def counting_queue_peek(self):
    if len(self.queue) == 0:
        return None
    el, _ = self.queue[0]
    return el

CountingQueue.peek = counting_queue_peek


In other words, we first create a function (in this case `counting_queue_peek`) and then we assign it to the method `peek` of `CountingQueue`.  It's a bit unusual, but it works, and it relieves us from the task of redefining the class each time we need a new method. 

In [ ]:
q = CountingQueue()
q.add("cat")
q.add("dog")
q.peek()


'cat'

### `__len__`

In [ ]:
def counting_queue_len(self):
    lenth = 0
    for k, v in self.queue:
        lenth += v
    return lenth
 
CountingQueue.__len__ = counting_queue_len


In [ ]:
# 5 points.  Simple tests

q = CountingQueue()
assert len(q) == 0
q.add("cat")
q.add("dog")
assert len(q) == 2


In [ ]:
# 5 points.  More complicated tests.

q = CountingQueue()
assert len(q) == 0
q.add("cat")
q.add("cat")
assert len(q.queue) == 1
assert len(q) == 2
q.add("dog")
assert len(q) == 3
assert len(q.queue) == 2
q.add("dog")
assert len(q) == 4
assert len(q) == 4 # Hey, just in case you went for the quantum-mechanical solution ;-)
assert len(q) == 4
assert len(q.queue) == 2


In [ ]:
# 5 points.  Works same as Queue.

import random

for k in range(100):
    q0 = Queue()
    q1 = CountingQueue()
    for _ in range(100):
        el = random.choice(["a", "b", "c"])
        q0.add(el)
        q1.add(el)
        assert len(q0) == len(q1)
        assert len(q0.queue) >= len(q1.queue)


### `__iter__`

In [ ]:
def counting_queue_iter(self):
    # YOUR CODE HER
    for i in self.queue:
        for _ in range(i[1]):
            yield i[0]

CountingQueue.__iter__ = counting_queue_iter


In [ ]:
# 5 points.  Simple tests.

q = CountingQueue()
q.add("cat", count=2)
q.add("dog", count=3)
assert [x for x in q] == ["cat"] * 2 + ["dog"] * 3
print([x for x in q] == ["cat"] * 2 + ["dog"] * 3)
print([x for x in q])


True
['cat', 'cat', 'dog', 'dog', 'dog']


In [ ]:
# 5 points.  Works the same as queue.

for k in range(100):
    q0 = Queue()
    q1 = CountingQueue()
    for _ in range(100):
        el = random.choice(["a", "b", "c"])
        q0.add(el)
        q1.add(el)
        assert [x for x in q0] == [x for x in q1]


### `__in__`

In [ ]:
def counting_queue_in(self, el):
    # YOUR CODE HERE
    # if el in self.queue return True else return False 
    return True if el in self.queue else False

    """
    if el in self.queue:
        return True
    else:
        return False
    """


CountingQueue.__in__ = counting_queue_in


In [ ]:
# 0 points.  Simple cases.

q = CountingQueue()
assert "cat" not in q
q.add("cat", count=2)
assert "cat" in q
assert "dog" not in q
q.add("dog")
assert "cat" in q
assert "dog" in q
q.get()
assert "cat" in q
assert "dog" in q
q.get()
assert "cat" not in q
assert "dog" in q
q.get()
assert "cat" not in q
assert "dog" not in q


In [ ]:
# 0 points.  Behaves the same as Queue.

elements = range(5)
for k in range(100):
    q0 = Queue()
    q1 = CountingQueue()
    for _ in range(20):
        el = random.choice(elements)
        q0.add(el)
        q1.add(el)
        for x in elements:
            assert (x in q0) == (x in q1)


### `__getitem__`

In [ ]:
def counting_queue_getitem(self, n):
    # YOUR CODE HERE
    x = []
    for i in self.queue:
        for _ in range(i[1]):
            x.append(i[0])
    return x[n]


CountingQueue.__getitem__ = counting_queue_getitem


In [ ]:
# 5 points: simple tests.

q = CountingQueue()
q.add("cat", count=2)
q.add("dog", count=3)
q.add("bird", count=4)
els = [q[i] for i in range(9)]
print(els)
assert els == ['cat'] * 2 + ['dog'] * 3 + ['bird'] * 4
# Let's do it again.
els = [q[i] for i in range(9)]
assert els == ['cat'] * 2 + ['dog'] * 3 + ['bird'] * 4


['cat', 'cat', 'dog', 'dog', 'dog', 'bird', 'bird', 'bird', 'bird']


In [ ]:
# 5 points: you raise IndexError when accessing elements out of bounds.

q = CountingQueue()
q.add("cat", count=2)
q.add("dog", count=3)

# Raise IndexError when it's too high...
try:
    q[5]
    assert False, "Failed to raise IndexError"
except IndexError:
    pass
assert q[4] == "dog"
try:
    q[5]
    assert False, "Failed to raise IndexError"
except IndexError:
    pass

# And also when it's too low.
try:
    q[-10]
    assert False, "Failed to raise IndexError"
except IndexError:
    pass

# or too large:
try:
    q[10]
    assert False, "Failed to raise IndexError"
except IndexError:
    pass

# And raise TypeError if you try to index with a non-integer.
try:
    q["hello"]
    assert False, "Failed to raise TypeError"
except TypeError:
    pass



In [ ]:
# 5 points.  Behaves the same as Queue.

elements = range(3)
for k in range(100):
    q0 = Queue()
    q1 = CountingQueue()
    for m in range(40):
        el = random.choice(elements)
        q0.add(el)
        q1.add(el)
        for i in range(m):
            assert q0[i] == q1[i]


q = CountingQueue()
q.add("cow", count=4)
q.add("turtle", count=5)

# Raise IndexError when it's too high...
try:
    q[10]
    assert False, "Failed to raise IndexError"
except IndexError:
    pass
assert q[8] == "turtle"
try:
    q[9]
    assert False, "Failed to raise IndexError"
except IndexError:
    pass

# And also when it's too low.
try:
    q[-20]
    assert False, "Failed to raise IndexError"
except IndexError:
    pass

# And raise TypeError if you try to index with a non-integer.
try:
    q["cow"]
    assert False, "Failed to raise TypeError"
except TypeError:
    pass